---
title: 日時モジュールの使い方
subject: python
keywords: [python, pendulum, time, datetime]
authors: [Shota Takahashi]
exports:
  - format: pdf
---

+++ {"part": "abstract"}
Python標準モジュールの``time``と``datetime``、
そして外部モジュールの``pendulum``を使って、
よくある日時の取り扱いを比べてみました。
+++

``time``と``datetime``は標準モジュールなので、そのまま読み込めます。
``pendulum``は``pip``などを使ってあらかじめ追加しておきます。
今回は最新版のv3系を使いました。

In [2]:
import time
import datetime
import pendulum
pendulum.__version__

'3.0.0'

# 現在時刻を取得したい

スクリプトを実行した時刻など、現在時刻を取得したい場合はよくあります。

1. ``datetime.datetime.now``
2. ``pendulum.now``
3. ``pendulum.from_timestamp``

In [29]:
dt = datetime.datetime.now()
print(dt)
dt

2024-01-26 15:24:20.030585


datetime.datetime(2024, 1, 26, 15, 24, 20, 30585)

``datetime.datetime.now``の引数に``datetime.timezone``を指定すると、タイムゾーン情報を持たせることができます。

In [30]:
dt = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
print(dt)
dt

2024-01-26 15:24:28.871029+09:00


datetime.datetime(2024, 1, 26, 15, 24, 28, 871029, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

``pendulum``はデフォルトでタイムゾーン情報を持った日時オブジェクトを作成します。

In [31]:
dt = pendulum.now()
print(dt)
dt

2024-01-26 15:24:41.967710+09:00


DateTime(2024, 1, 26, 15, 24, 41, 967710, tzinfo=Timezone('Asia/Tokyo'))

``time.time()``で現在のUNIX時間を取得できます。
``pendulum.from_timestamp``を使って``DateTime``オブジェクトに変換できます。

In [7]:
ut = time.time()
pendulum.from_timestamp(ut)

DateTime(2024, 1, 26, 6, 1, 39, 554112, tzinfo=Timezone('UTC'))

``datetime.datetime.now``で取得した日時オブジェクトは、``pendulum.instance()``で``DateTime``オブジェクトに変換できます。
もとの日時オブジェクトがタイムゾーン情報を持たない場合は``UTC``に変換されます。
タイムゾーン情報を持つ場合は、時間差情報が追加されます。

In [33]:
dt = datetime.datetime.now(datetime.timezone(datetime.timedelta(hours=9)))
pendulum.instance(dt)

DateTime(2024, 1, 26, 15, 27, 42, 885222, tzinfo=FixedTimezone(32400, name="+09:00"))

# 任意の日付したい

1. ``datetime.datetime``
2. ``pendulum.datetime``
3. ``pendulum.local``

``年, 月, 日, 時, 分, 秒``のタプルを与えて、任意の日時オクジェクトを作成できます。
``datetime``モジュールはタイムゾーン情報を持たないため、``tz-naive``なオブジェクトが作成されます。

In [34]:
dt = datetime.datetime(2024, 1, 2, 3, 45, 56)
print(dt)
dt

2024-01-02 03:45:56


datetime.datetime(2024, 1, 2, 3, 45, 56)

同様のタプルを``pendulum.datetime``に与え、タイムゾーン情報を持った``tz-aware``なオブジェクトが作成できます。
デフォルトでは``UTC``が設定されます。``tz="Asia/Tokyo``オプションを追加すると、日本時刻として作成できます。

In [35]:
dt = pendulum.datetime(2024, 1, 2, 3, 45, 56)
print(dt)
dt

2024-01-02 03:45:56+00:00


DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('UTC'))

``pendulum.local``を使うと、実行環境のタイムゾーンを持った日時オブジェクトが作成できます。

In [36]:
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
print(dt)
dt

2024-01-02 03:45:56+09:00


DateTime(2024, 1, 2, 3, 45, 56, tzinfo=Timezone('Asia/Tokyo'))

# 日付を変換したい

ある日時オブジェクトを、別の並びの文字列に変換したいことがあります。
たとえばRSSフィードの日時は``曜日, 日, 月, 年 時:分:秒 +Z``であったり、
ISO8601形式の日時は``年-月-日T時:分:秒+Z``であったりします。

``datetime``モジュールの場合、変換後の文字列は自分で指定する必要があります。
``pendulum``パッケージの場合、よく使われる日時の文字列はプリセットされています。
なるべくプリセットされたものを使うのがよいでしょう。

In [39]:
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
dt.to_rss_string()

'Tue, 02 Jan 2024 03:45:56 +0900'

In [40]:
dt = pendulum.local(2024, 1, 2, 3, 45, 56)
dt.to_iso8601_string()

'2024-01-02T03:45:56+09:00'

# 日付を読み取りたい

``pendulum.parse``で一般的な日付フォーマットをパースできます。
タイムゾーンが指定されていない場合は、``UTC``になります。

In [10]:
pendulum.parse("2022-09-26T17:00:00")

DateTime(2022, 9, 26, 17, 0, 0, tzinfo=Timezone('UTC'))

``tz``オプションを使って、タイムゾーンを明示してパースできます。

In [7]:
pendulum.parse("2022-09-26T17:00:00", tz="Asia/Tokyo")

DateTime(2022, 9, 26, 17, 0, 0, tzinfo=Timezone('Asia/Tokyo'))

# 独自形式の日付を読み取りたい

``pendulum.from_format``で独自形式の日付をパースできます。
利用可能な日付トークンは https://pendulum.eustace.io/docs/#tokens を参照してください。

In [26]:
pendulum.from_format("01/Sep/2012:06:05:11 +0000", "DD/MMM/YYYY:HH:mm:ss ZZ")

DateTime(2012, 9, 1, 6, 5, 11, tzinfo=Timezone('+00:00'))

In [21]:
pendulum.from_format("26/Sep/2022:06:05:11 +0900", "DD/MMM/YYYY:HH:mm:ss ZZ")

DateTime(2022, 9, 26, 6, 5, 11, tzinfo=Timezone('+09:00'))

# ISO8601形式に変換する

``isoformat``を使うと、ISO8601形式（``YYYY-mm-ddT%HH:MM:SS+Z``）に簡単に変換できます。
また、さまざまな表示形式に出力できます。

In [22]:
dt = pendulum.parse("2022-09-26T17:00:00")
dt.isoformat()

'2022-09-26T17:00:00+00:00'

In [23]:
dt = pendulum.from_format("26/Sep/2022:06:05:11 +0900", "DD/MMM/YYYY:HH:mm:ss ZZ")
dt.to_iso8601_string()

'2022-09-26T06:05:11+09:00'